In [1]:
from os.path import join as pjoin
import numpy as np
from nilearn.image import load_img
from nilearn.masking import intersect_masks
from nipype import config
from nipype.interfaces.fsl.maths import TemporalFilter
from nipype.interfaces.fsl.model import MELODIC
from nipype.interfaces.fsl.preprocess import SUSAN
from nipype.interfaces.utility import Function
from nipype.pipeline.engine import MapNode, Workflow, Node
from tqdm.notebook import tqdm

220414-10:08:38,648 nipype.utils WARNING:
	 A newer version (1.7.1) of nipy/nipype is available. You are using 1.5.1


In [2]:
config.enable_debug_mode()

In [4]:
def calc_susan_thresh(boldfile, maskfile, timeax=0, median_factor=.75):
    """
    Calculate the median value within brainmask and multiply with fixed factor to get an estimate of the contrast
    between background and brain for FSL's SUSAN.
    """
    from nilearn.masking import apply_mask
    import numpy as np
    data = apply_mask(boldfile, maskfile)
    med = np.median(data.mean(axis=timeax))
    del data  # suspect memory leak
    return med * median_factor

In [3]:
# TODO: Do preprocessing and ICA

# Required inputs:
# - functional run (bold file)
# - brain mask

# Desired outputs:
# - A nipype workflow that combines precessing (smoothing/temporal filtering) 
# and ICA for one functional run

In [ ]:
def make_subject_ica_wf(bold_file, mask_file, hpf_sec, tr):
    """
    TODO: give TR as input (to this function) or infer from data?
    """
    ### TODO: create input spec ##
    
    # create node for smoothing
    calcthresh = Node(
        Function(
            function=calc_susan_thresh, input_names=['boldfile', 'maskfile'], 
            output_names=['smooth_thresh'], 
            name='calcthresh')
    )
    susan = Node(SUSAN(fwhm=fwhm), name='susan') # requires 
    # ... temporal filtering
    tfilt = Node(TemporalFilter(highpass_sigma=float(hpf_sec/tr)), name='tfilt')
    # ... ICA
    # inputs: out_dir, no_bet=???, num_ICs???, 
    melodic = Node(MELODIC(tr_sec=tr, out_all=True, no_bet=True, report=True), name='melodic')

    ### TODO: create output spec ##
    
    # connect nodes in workflow
    wf = Workflow(name='melodicwf', base_dir='/home/ocontier/scratch'
    wf.connect([
    #   (inputspec, ..., ...)
        (calcthresh, susan, [('smooth_thresh', 'brightness_threshold')]),
        (susan, tfilt, [('smoothed_file', 'in_file')]),
        (tfilt, melodic, [('out_file', 'in_files')]),
    #   (..., outputspec, ...)
    ])
    return wf

# Tasks

- Add inputspec and outputspec to our workflow!
- execute for a bunch of runs/subjects/datasets!
- Put the code we wrote today in a python file in our repo for future happy uses!!!